## LangChain Agent + Speech Recognition

In [1]:
import whisper
from langchain.tools import Tool
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
import gradio as gr

In [2]:
whisper_model = whisper.load_model("base")
def transcribe_audio(file_path):
    print("DEBUG: file_path =", file_path, type(file_path))
    result = whisper_model.transcribe(file_path)
    return result["text"]

In [3]:
def rag_qa(query):
    return rag_pipeline(query)
qa_tool = Tool(
    name="YouTubeQA",
    func=rag_qa,
    description="Answer questions about YouTube videos using RAG."
)

In [4]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo")
tools = [qa_tool]
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True
)

C:\Users\Mercy\AppData\Local\Temp\ipykernel_17520\3760734133.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-3.5-turbo")
C:\Users\Mercy\AppData\Local\Temp\ipykernel_17520\3760734133.py:3: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <http

In [ ]:
def process_audio_with_agent(audio_file):
    text_query = transcribe_audio(audio_file)
    answer = agent.run(input=text_query)
    return f"Q: {text_query}\n\nA: {answer}"

demo = gr.Interface(
    fn=process_audio_with_agent,
    inputs=gr.Audio(type="filepath", label="Record or Upload Audio"),
    outputs=gr.Textbox(label="Q & A", lines=6),
    title="ServiceNow QA Assistant: Ask by Voice!",
    description="The answer will appear below your transcribed question."
)
demo.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


DEBUG: file_path = C:\Users\Mercy\AppData\Local\Temp\gradio\aefc7cc471dab426274c54cc64f601b27774d5cbe36df096c8d6c6d695c06f3b\audio.wav <class 'str'>


c:\Users\Mercy\.pyenv\pyenv-win\versions\3.10.11\lib\site-packages\whisper\transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
C:\Users\Mercy\AppData\Local\Temp\ipykernel_17520\2344364971.py:3: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  answer = agent.run(input=text_query)




> Entering new AgentExecutor chain...
ServiceNow is a cloud-based platform that provides IT service management (ITSM), as well as digital workflow automation. It helps organizations manage their IT services, automate routine tasks, and streamline business processes. ServiceNow offers a wide range of tools and applications to help businesses improve efficiency, productivity, and customer satisfaction.

> Finished chain.


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "c:\Users\Mercy\.pyenv\pyenv-win\versions\3.10.11\lib\site-packages\uvicorn\protocols\http\h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
  File "c:\Users\Mercy\.pyenv\pyenv-win\versions\3.10.11\lib\site-packages\uvicorn\middleware\proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
  File "c:\Users\Mercy\.pyenv\pyenv-win\versions\3.10.11\lib\site-packages\fastapi\applications.py", line 1054, in __call__
    await super().__call__(scope, receive, send)
  File "c:\Users\Mercy\.pyenv\pyenv-win\versions\3.10.11\lib\site-packages\starlette\applications.py", line 112, in __call__
    await self.middleware_stack(scope, receive, send)
  File "c:\Users\Mercy\.pyenv\pyenv-win\versions\3.10.11\lib\site-packages\starlette\middleware\errors.py", line 187, in __call__
    raise exc
  File "c:\Users\Mercy\.pyenv\pyenv-win\versions\3.10

DEBUG: file_path = C:\Users\Mercy\AppData\Local\Temp\gradio\5ce6baffdd050984bdc27d17b4960e91fdd51999d4709b0a7cd4204db2c97338\audio.wav <class 'str'>


c:\Users\Mercy\.pyenv\pyenv-win\versions\3.10.11\lib\site-packages\whisper\transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")




> Entering new AgentExecutor chain...
In ServiceNow, AI functionalities are used to automate and streamline various processes within the platform. Some common AI functionalities in ServiceNow include:

1. Virtual Agents: AI-powered chatbots that can assist users with common IT service requests, such as password resets, incident reporting, and knowledge base searches.

2. Predictive Intelligence: AI algorithms that analyze historical data to predict potential issues, such as outages or equipment failures, before they occur.

3. Natural Language Understanding: AI technology that enables users to interact with the ServiceNow platform using natural language queries, making it easier to search for information or create tickets.

4. Machine Learning: AI algorithms that can learn from data and make predictions or recommendations based on patterns and trends, such as identifying recurring issues or anomalies in IT operations.

5. Intelligent Automation: AI-driven automation capabilities that

In [ ]:
import gradio as gr

def process_query(text_query, audio_file, history):
    if audio_file is not None:
        text_query = transcribe_audio(audio_file)
        question = text_query
    elif text_query and text_query.strip():
        question = text_query.strip()
    else:
        return history + [("User", "Please enter a question or upload audio.")]
    answer = agent.run(input=question)
    history = history + [(f"User: {question}", f"Agent: {answer}")]
    return history

with gr.Blocks() as demo:
    gr.HTML("<h1 style='text-align: center;'>ServiceNow QA Agent</h1>")
    gr.Markdown("<center>Type or record your question below. The bot will provide you answer</center>")
    chatbot = gr.Chatbot(label="Conversation")
    with gr.Row():
        text_input = gr.Textbox(label="Type Your Question", lines=2)
        audio_input = gr.Audio(type="filepath", label="Or Record/Upload Audio")
    submit = gr.Button("Submit")
    #clear = gr.Button("Clear Chat")

    state = gr.State([])  # to hold chat history

    submit.click(
        process_query,
        inputs=[text_input, audio_input, state],
        outputs=chatbot
    ).then(
        lambda history: history,  # update state with latest history
        inputs=chatbot,
        outputs=state
    )
    #clear.click(
        #lambda: [],
        #None,
        #[chatbot, state]
    #)

demo.launch(share=True)


C:\Users\Mercy\AppData\Local\Temp\ipykernel_17520\3495208296.py:18: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="Conversation")


* Running on local URL:  http://127.0.0.1:7873
* Running on public URL: https://cbd9e59567dff68429.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)




> Entering new AgentExecutor chain...
ServiceNow is a cloud-based platform that provides IT service management (ITSM), as well as other service management solutions for various departments within an organization. It helps organizations streamline their operations, automate workflows, and improve efficiency by providing tools for service desk management, incident management, change management, and more. ServiceNow is widely used by businesses to optimize their service delivery and enhance customer satisfaction.

> Finished chain.
